## Instalacion de librerias de langchain

In [1]:
%pip install --upgrade --quiet langchain langchain-openai langchain_google_community langchain_community langchainhub pypdf faiss-cpu tiktoken google-search-results wikipedia


Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-pinecone 0.1.3 requires langchain-core<0.3,>=0.1.52, but you have langchain-core 0.3.8 which is incompatible.


In [2]:
import os
import langchain
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate #Funcion para crear prompt con template
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

langchain.debug = True
os.environ["OPENAI_API_KEY"] = ""
 .

## Subida de documentos, embedding y base de datos vectorial

In [3]:
loader = PyPDFLoader("C:/Users/Seba/capstone/desarrollo/backend/api/celula_unidad_funcional_de_los_seres_vivos.pdf")
pages = loader.load_and_split()

# Funcion para crear embeddings
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
)

# Creamos una base de datos vectorial a partir de las páginas del documento
faiss_index = FAISS.from_documents(pages, embeddings)
retriever = faiss_index.as_retriever()

## Creacion del modelo LLM con mi Template Prompt

In [125]:
template = """Eres un un profesor experto Ciencias de la prueba PAES, PTA, PSU de Chile, y los alumnos vienen a solicitarte preguntas de 5 alternativas para poder poner a prueba sus conocimientos. Entrega la pregunta y respuesta en formato json, asi:
pregunta": "Texto de la pregunta",
                "opciones": [
                    {{"texto": "Opción 1", "es_correcta": none}},
                    {{"texto": "Opción 2", "es_correcta": none}},
                    {{"texto": "Opción 3", "es_correcta": none}},
                    {{"texto": "Opción 4", "es_correcta": none}},
                    {{"texto": "Opción 5", "es_correcta": none}}
                ]
Usa la siguiente informacion para elavorar tus preguntas.
{context}.
Solo entrega el json.

Pregunta: {question}
"""

### Creacion de Tool para distinta materia

#### Esto permite que en cada tool se alimente de distintos documentos al realizar la pregunta

In [126]:
from langchain.tools.retriever import create_retriever_tool
from langchain_core.prompts import MessagesPlaceholder
from langchain.agents import AgentExecutor, create_openai_tools_agent

In [127]:
retriever_tool = create_retriever_tool(
    retriever, #No olvidar que este retriver tiene solo el documento de las celulas
    "Materia_Ciencias_Biologia",
    "Usa este recurso para preguntas que correspondan al campo de la Biologia.",
)

tools = [retriever_tool]

In [143]:
model = ChatOpenAI(temperature=0.3, model="gpt-4o", seed=211)
agent = create_openai_tools_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [1]:
#Lista de herramientas, cono solo una, el documento de la celula, por ahora
tools

NameError: name 'tools' is not defined

In [133]:
user_input = "Cual es el mayor secreto de harry potter?"
context_info = ""
# Ejecuta la consulta usando invoke
response = agent_executor.invoke({"input": user_input, "context": context_info})

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Cual es el mayor secreto de harry potter?",
  "context": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] s] Exiting Chain ru

## Creacion de Tools personalizado

In [134]:
from langchain_core.tools import tool

@tool
def secretos_harry_potter() -> str:
    """Entrega el mayor secreto de Harry Potter."""
    return "El mayor secreto de Harry Potter es que sueña con empanadas de pino."


#### Actualizamos tools y agente

In [137]:
tools = [retriever_tool,secretos_harry_potter]
agent = create_openai_tools_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [144]:
#Aseguramos que la nueva informacion de ingreso como herramienta, y aun sigue tomando el template determinadom anteriormente
user_input = "Harry potter tiene algun secreto? si o no"
context_info = ""
# Ejecuta la consulta usando invoke
response = agent_executor.invoke({"input": user_input, "context": context_info})

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Harry potter tiene algun secreto? si o no",
  "context": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] s] Exiting Chain ru